# Part II: My innovative idea: PMG
This is the second part of a binary classification task to predict renewal of customers which explained in the first part.

# Abstract
I will introduce my innovative approach, Parallel Modeling of Groups or PMG which is useful when data are derived from multiple sources or from the one source with different modes. After explaining theory behind that approach, I will implement PMG, to improve the performance of modeling of static datasets that of implicitly or explicitly consisting of unknown groups and 2 versions of PMG, PMG-clustered and PMG-boosted will be explained and applied on current dataset; then a deeper version of PMG along with a new hybrid architecture of combining deep PMG with boosting and feature bagging will be explained. The results do not show significant difference since this dataset does not meet requirements as it will be explained; however, I believe, the presented deep PMG algorithm will play an essential role in the future of ML on special cases that meet explained conditions.

# Table of Contents
1. Introduction
2.PMG: Parallel Modeling of Groups
3. Import & Read & Assigning Initial Values
4. Functions I
5. PMG Algorithm
6. Functions II
7. Implementation of PMG-2TF (PMGB, PMG-Boosted, or PMG-TF with 2 Groups)
8. Implementation of PMG-3TF (PMGB, PMG-Boosted, or PMG-TF with 3 Groups)
9. Implementation of PMGC (PMG-Clustered or General PMG) with K Groups
10. PMG: Drawbacks, Challenges and Conditions
11. Another Viewpoint: Optimization Perspective
12. What Distinguish PMG from Other Methods in Literature?
13. Advantages of PMG
14. Conclusion
15. Future Works: Hybrid PMG with Boosting & Bagging

# 1. Introduction
# 1.1 Short Introduction
This is a severely class-imbalanced dataset, described at the first part of notebook at my GitHub, make it difficult to reach high F-score. We presented pipelines of handling this dataset including data preprocessing, feature engineering, exploratory data analyzing, modeling and evaluation in the first part of notebook. Here, in the second part, we presented an innovative idea, Parallel Modeling of Groups (PMG). However, PMG is not limited to this dataset and I believe it is a revolution in Machine learning algorithms that will tremendously superior to many benchmark algorithms on some datasets. It is necessary to emphasize that PMG is not appropriate when dataset does not meet special conditions that will be explained in “challenges and conditions for applying PMG” section, in this notebook. 

# 1.2 Data Types
It is vital to recognize data type before diving into modeling step. There are different ways to categorize data; from one view point, dependency of output on the previous outputs, data can be categorized to:
a) static (non-dynamic): Data is not change or in not time-dependent. (most cases from NLP to Image recognition)
b) Time Series: Data collected at regular intervals over time: weather prediction, website traffic data

Many tasks can be solved via algorithms corresponded to the above categories; but, still, you cannot answer to all tasks with them for example every time-variant system is not necessarily a time series and there are other scenarios:
a) cross-sectional data: Data driven from many different individuals at a given time, each observation belonging to a different individual.
b) time-variant data that are not time series: Data is dynamic and changes over time but it is hard to predict that change in time because in this type, unlike time series, the system that produces data might significantly change over time or the system changes but output is not necessarily dependent on previous outputs; Therefore, despite some similarities, they are not the same and need different solutions. For example, adaptive algorithms can be used in anomaly analysis and fraud detection systems to continuously learn and adapt to new fraud patterns.
c) Panel data (cross-sectional time-series): Data collected from multiple systems over time and include cross-sectional and time-series dimensions like academic performance of students of a class from several viewpoints over 4 years

# 2. PMG: Parallel Modeling of Groups
# 2.1 Why PMG?
While there are still many other scenarios like combination of mentioned categories, PMG is designed to address one category where dataset is static and does not change over time but consist of multiple groups with different properties that are originally driven from multiple data sources. Before this notebook, these datasets are addressed like other static datasets which is not appropriate. Can ML algorithms find a function to map a dataset when target label is consisting of 2 irrelevant phenomena, you say speed of wings of a butterfly in South Africa and gold price in USA simultaneously? A Well-designed ML algorithm is able to find such a model but in the cost of decreasing in evaluation metric and increasing the model complexity. 

# 2.2 Solution
Sometimes, there are groups in dataset representatives of multiple phenomena that require separate processing approaches. Although it is not impossible to develop accurate model, such complicated models are prone to overfit to map data from all sources together in one. Here, I propose a simple but innovative powerful approach, finding an independent model for each group, to improve the predefined evaluation metric of modeling on such datasets where data originally come from different sources. First divide samples to separate groups then find an appropriate model for each group. The key idea behind grouping is to leverage the similarity between each group to improve learning process in such a way that model can potentially capture patterns more effectively, leading to better generalization and performance. Although domain knowledge is not vital for implementation of PMG, it plays an essential role in simplicity of implementation of this approach, especially in the number of groups in dataset. It is important to remind that there are 2 different scenarios in PMG cases. On one hand, there are datasets that the group label is available in features. Although in these cases one model is enough since even without separate models, more or less, that feature can participate in the model to map that difference in behaviors, it is still better to model each group separately, especially if the difference between their behaviors is significant. On the other hand, there are datasets that the group label is not collected. PMG is able to address this scenario by estimating the number of groups and group members before modeling each group.

# 2.3 Different Outputs for the Same Inputs in Static Systems
Sometimes target of static systems shows different behavior in the same situation i.e. Y is different for the same or similar X values in different samples. Can you guess why? We will explain some reasons. 
a) randomness: The First common reason is intrinsic randomness of target variable.
b) Bad Data: The second common reason might be raised due to noise or any type of uncertainty in dataset that result in incorrect, incomplete, outdated, or irrelevant data. 
c) Incomplete Features: The third scenario is that data are collected for unknown phenomena and the features are not necessarily informative being correlated with target variable; for example, a study about probability of occurrence of an earthquake in a city. This scenario is might happens even in scenarios that scientifically well recognized but collected features are not enough to explain the data, for example number of features are too low to describe the performance of the system.
d) Multiple Well-Known Modes: The fourth scenario happens when the collected data gathered from a source have multiple modes while these modes are known and they are gathered and available at features in dataset. Consider a dataset for studying the effect of blood minerals and hormone levels on special cancer. ML algorithms can solve the problem but we know that hormonic system of men and woman are significantly different. Isn't it much better to divide data into two parts; Men and women or, for example, into 4 parts including men, women, non-binaries and children then finding much simpler and more accurate ML algorithms for each group? 
e) Multiple Unknown Modes: The fifth scenario happens when the collected data gathered from a source have multiple modes while these modes are scientifically unknown or they are not collected. What if mentioned dataset for cancer do not include sex feature? We can still decide to divide data into 2 parts because, from domain knowledge, we know people are men or women. However, the grouping error will be inevitable. What if we have no prior knowledge about how many groups are data? As another example, consider Diabetes Dataset to predict whether somebody suffers from diabetes or not. We can develop a model with good accuracy or any other metrics like f-score; however, there is a limit for improving evaluation metric even in such a static dataset due to noise in data, lack of some important features in dataset and etc. But what if the main reason of this limit is that data driven from multiple sources. we know there are several diabetic issues like type-I, type-II, Cystic fibrosis, Gestational, Monogenic diabetes syndromes, Post transplantation diabetes mellitus and etc. that have very different behaviors and properties some of which can be considered as completely different illness. In fact, before working on current dataset, I worked on Kaggle Diabetes Dataset, another imbalanced binary classification dataset, that was my inspiration to reach the idea of MPG when working on current dataset. Isn’t it funny, I am late, lol! I think this approach is more appropriate for that dataset which consists of different groups based on domain knowledge as mentioned or on other similar datasets rather than trying on current dataset. We may try it in the future.
f) Multiple Sources: The sixth scenario happens when data are collected from multiple sources with different response despite some similarities in these sources. Consider an image dataset for classification of numbers from 0 to 9, consisting of two different patterns, for example composed of MNIST where numbers are written in white color with black background and another dataset where numbers is written in black with white background. As a first solution, we can train the model to classify data into 20 categories instead of 10. However, in this special case, it is not very hard to handle the task, for example we can transform all data to black background with some tricks even if there is no feature implying that data are driven from which dataset. However, it is not always that much straightforward; assume that there are more differences in datasets make it impossible to transform one type to the other one. Isn't it wise to first apply simple algorithm like k-means to separate data driven from 2 datasets, if possible, then applying separate model for each group? we assumed that data are driven from 2 datasets; what if we have no prior information about number of sources? It is harder but still can be estimated either via clustering algorithms that automatically determined the number of clusters or via other ways like elbow method. 

# 2.4 Applicable Scenarios
PMG is the solution for the fourth, fifth and sixth scenarios when data gathered from multiple sources or behave in multiple modes. Although available features of fourth scenario will help ML classic models, PMG is able to reach better results; however, PMG will show its power with a big improvement in fifth and sixth scenarios when there is no feature in dataset to explain that behavior.

# 3. Import & Read & Assigning Initial Values

In [1]:
# 3. import & read & initial values
# 3.1 import 
import pandas as pd; import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense as kD
from keras.regularizers import L1L2

from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier as ABC, GradientBoostingClassifier as GBC, RandomForestClassifier as RFC
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score, confusion_matrix, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# 3.2 read
add = r'F:\My23Codes\To Git\4_insurance\git\extracted_data'
f_name_train, f_name_test = r'\traindata_sia_train.csv', r'\traindata_sia_test.csv'
f_sia_train, f_sia_test = add + f_name_train, add + f_name_test
# note the trainin and test set of this code is all part of the labeled traing set of kaggle dataset 
train0 = pd.read_csv(f_sia_train);  train0.index = train0["id_temp"]; train0.drop("id_temp", axis=1, inplace = True)
test0 = pd.read_csv(f_sia_test);    test0.index = test0["id_temp"];   test0.drop("id_temp", axis=1, inplace = True) 
train, test = train0.copy(), test0.copy()

# 3.3 Assigning Initial Values
# short names for cols & split data 
tar, tar2 = "target", "new_idea"               # traget label for modeling & # temporary traget label for grouping
X_train0, y_train0, X_test0, y_test0 = train.drop(tar, axis=1), train[tar], test.drop(tar, axis=1), test[tar]
X_train, y_train, X_test, y_test = X_train0.copy(), y_train0.copy(), X_test0.copy(), y_test0.copy()

# 3.4 feature selection for modeling and grouping
# col_g: index number of features for grouping (indexes: 0:55 ==> poly   60:65  ==> corresponding PCA_p & lda_p for poly) 
# col_m: the remaining indices of cols will be used for modeling to achieve more robust independent steps
col_g = list(range(55)) + list(range(60,65))           # selected cols (features) for grouping
col_m = list(set(range(X_train.shape[1]))-set(col_g)) # selected cols for modeling   # note: X_train.shape[1]==72

# 4. Functions I

In [2]:
# 4. functions I
# 4.1 functions I (general functions)
def rnd(a, decimals=2):
    return np.round(a, decimals=decimals)

def percent0_cond(cond1, cond2 = pd.Series("all"), df = train):
    if (cond2=='all').any():
         cond2 = [True for i in range(len(df)) ]
    return rnd(100*len(df[cond1])/len(df[cond2]))

def percent0(df):
    return rnd(100*sum(df)/len(df))

def sorted_corrwith(X,y):
    idx = np.abs(X.corrwith(y)).sort_values( ascending=False).index
    return pd.DataFrame(X[idx].corrwith(y)).T

def my_F1_score(TP, FN, FP):
    # needed for calculation of F-score in my method (2 parallel model)
    Precision = TP / (TP+FP)
    Recall = TP / (TP + FN)
    my_F1_Score = 2 * (Precision * Recall) / (Precision + Recall)
    print("Precision, Recall & F1_score: ", rnd(100*Precision), rnd(100 * Recall ), rnd(100*my_F1_Score) )
    
# 4.2 functions II (arguments & FE)
def dic2(i): 
    # for adjusting class_weight
    return {0:i, 1:1-i} 
def dic3(i, j=0.035):
    # for adjusting class_weight
    return {0:i, 1:1-i-j, 2:j}

def LDA_n(X_train , y_train ,  X_test , y_test , n=1):
    # FE: n ==> number of extracted features
    lda = LinearDiscriminantAnalysis(n_components = n)
    X_train_LDA = lda.fit_transform(X_train, y_train)
    X_test_LDA = lda.transform(X_test)    
    return X_train_LDA, X_test_LDA

# 4.3 functions (modeling)
def LR(d, n_classes = 2, j = 0.035):
    if n_classes == 3:
        return LogisticRegression(class_weight = dic3(d, j), C=1, penalty='l2', max_iter=300)
    else:
        return LogisticRegression(class_weight = dic2(d), C=1, penalty='l2', max_iter=300 )

def RF(d, n_classes = 2, j = 0.035):
    if n_classes == 3:
        return RFC(class_weight=dic3(d, j),max_depth=3, max_leaf_nodes=17, max_features=9, random_state=0)
    else:
        return RFC(class_weight=dic2(d),max_depth=3, max_leaf_nodes=17, max_features=9, random_state=0)
    
def SV(d, n_classes = 2, j = 0.035):
    if n_classes == 3:
        return SVC(kernel="rbf", gamma=0.5, C=1.0, class_weight = dic3(d, j))
    else:
        return SVC(kernel="rbf", gamma=0.5, C=1.0,class_weight=dic2(d))
    
def model_clf(model, X_train, y_train, X_test = X_test, y_test=y_test):
    # print: cm, acc & F1      return: model(for any potential application in future) & pred
    model.fit(X_train, y_train)
    y_pred_train, y_pred_test = model.predict(X_train),  model.predict(X_test)
    cm_train, cm_test = confusion_matrix(y_train, y_pred_train), confusion_matrix(y_test, y_pred_test)
    TP_train, FN_train, FP_train = cm_train[0,0], cm_train[0,1], cm_train[1,0]
    TP_test, FN_test, FP_test = cm_test[0,0], cm_test[0,1], cm_test[1,0]
    display(cm_train, cm_test)
    print('train accuracy is', rnd(100 * accuracy_score(y_train, y_pred_train))\
          , '\t\t test accuracy is',rnd(100 * accuracy_score(y_pred_test,y_test)))
    print('train f1_score is', rnd(100 * f1_score(y_train, y_pred_train, pos_label = 0)),\
          '\t\t test f1_score is', rnd(100 * f1_score(y_pred_test,y_test, pos_label = 0)))    
    return model, y_pred_train, y_pred_test


def model_clf_2(model, X_train, y_train, X_test = X_test, y_test=y_test, n_class=2):
    # print: cm, acc & F1        return: TP, FP, FN for my_F1_score function
    model.fit(X_train, y_train)
    y_pred_train, y_pred_test = model.predict(X_train),  model.predict(X_test)
    cm_train, cm_test = confusion_matrix(y_train, y_pred_train), confusion_matrix(y_test, y_pred_test)
    if n_class ==2:
        TP_train, FN_train, FP_train = cm_train[0,0], cm_train[0,1], cm_train[1,0]
        TP_test, FN_test, FP_test = cm_test[0,0], cm_test[0,1], cm_test[1,0]
    else:
        TP_train, FN_train, FP_train = cm_train[0,0], cm_train[0,1] + cm_train[0,2], cm_train[1,0] + cm_train[2,0]
        TP_test, FN_test, FP_test    = cm_test[0,0],  cm_test[0,1] + cm_test[0,2],   cm_test[1,0] + cm_test[2,0]
    display(cm_train, cm_test)
    print('train accuracy is', rnd(100 * accuracy_score(y_train, y_pred_train))\
          , '\t\t test accuracy is',rnd(100 * accuracy_score(y_pred_test,y_test)))
    print('train f1_score is', rnd(100 * f1_score(y_train, y_pred_train, pos_label = 0)),\
          '\t\t test f1_score is', rnd(100 * f1_score(y_pred_test,y_test, pos_label = 0)))    
    return TP_train, FN_train, FP_train, TP_test, FN_test, FP_test


def MLP(X_train, y_train, X_test, y_test, u1=30, u2=30, ep=30, opt = "adam", d = 0.83, eps =0.1):
    # initial value of u1 & u2 are small units for fast run ===> increase them for selected model to improve prediction
    model = keras.Sequential() 
    model.add(kD(X_train.shape[1]))
    model.add(kD(u1, activation="relu"))
    model.add(kD(u2, activation="relu"))
    model.add(kD(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, y_train, validation_split= .1, epochs=ep, verbose=0, class_weight=dic2(d))
    y_pred_train_continuos = model.predict(X_train, batch_size=64, verbose=0)
    y_pred_test_continuos  = model.predict(X_test, batch_size=64, verbose=0)
    y_pred_train = np.round(y_pred_train_continuos-eps)
    y_pred_test = np.round(y_pred_test_continuos-eps)
    print(classification_report(y_pred_train, y_train))
    print(classification_report(y_pred_test, y_test))
    return model, y_pred_train_continuos, y_pred_test_continuos

# 4.4 initial values 
num_groups = 2 # numbers of group into which data will be divided
oversampling = False

# initial models
initial_model = LR(.87)

# grouping models
s3_model_g2, s3_model_g3 = LR(.87), LR(.79, 3)

# parallel models
model_g2_G0, model_g2_G1, model_g3_G0, model_g3_G1, model_g3_G2 = LR(0.82), LR(0.85),  LR(0.78,3), LR(0.75,3), LR(0.83,3)
# mdl_g2_g0, mdl_g2_g1 = LR(0.82), LR(0.85)
# mdl_g3_g0, mdl_g3_g1, mdl_g3_g2 = LR(0.78,3), LR(0.75,3), LR(0.83,3)

# 5. PMG Algorithm
# 5.1 Criteria for Dividing Samples into Groups
There is no general rule about how to groupify data and both unsupervised and supervised ML algorithms can be applied. The grouping method should be chosen based on the problem and nature of data but since my idea is new I have yet to think about how to find best grouping criteria which is not simple especially if there is no domain knowledge.  Data preprocessing, feature engineering and modeling of each group may be done independently. Two versions of grouping methods for PMG algorithm will be presented here: 

# 5.1.1  PMGC (PMG-Clustered or Parallel Modeling of Groups via Clustering) or General PMG
Generally, algorithms like hierarchical clustering and K-means seems to be great options since they can find possible clusters or groups which is the main reason of applying PMG. Elbow and average silhouette methods can be applied for finding number of clusters. Despite its more similarity to nature of idea, clustering is not the only option for finding groups and you may suggest your innovative classifiers. One of these innovative ideas is PMG-boosted.

# 5.1.2  PMG-boosted (Parallel Modeling of Groups via a Boosted-like Method) or PMG-TF (PMG-True False)
This approach is a little different from main idea of PMG since, unlike PMG-clustered, neither number of groups are determined by intrinsic characteristic of data nor the members are necessarily similar; however, it might be helpful in some dataset. Here, we groupify data based on mistakes of an initial simple model i.e. we will consider True predictions called G0 in one group and we will put misclassified samples called G1 in the other group. G0 group includes main part of data that can be easily model with initial model. Now, we will develop new model that to misclassified data that potentially come from different source. It is clear that this version does not include any boosting algorithm and do not work on residuals. I choose the name firstly because the approach is inspired from boosting and two-step classifications naively assuming that samples that are not truly predicted in the training set are comes from another source and secondly because it could be generalize as many times as needed like boosting-based algorithms that will be explained at next sections. Note that, initial model makes our training data labeled with G0 and G1 names and we should find a generalizable classifier to find the grouping label of test set before applying data to main models in each group. My underlying reason to develop PMG-boosted was that while we could predict main part of data correctly with a simple model, we could work more on misclassified subset of data to find new patterns. It is obvious that generalizability is a vital key in success of that version of algorithm. 

# 5.2 Feature Selection and Modeling
I use different features for grouping and modeling to leverage a simple version of feature bagging to reach a more robust model. However, the models of all subsets (except for class weight) are the same for main part of codes for simplicity. It would be much better to design special model including feature selection and designing the base model for each group.

# 5.3 The Applied Algorithm: Shallow PMG
For simplicity, since the idea is new, I did not try deeper layers at all in this notebook. Two version of algorithm is presented here:

# 5.3.1 PMG-Clustered (PMGC or General PMG or Original PMG)
This is the main idea that is explained and includes Parallel Modeling of Groups based on Clustering and steps can be shorten as follow:
step 1: Clustering data set into multiple groups 
step 2: labeling data based on clustering data (group1, group2, ...)
step 3: develop a model for predicting labels of group in step 2 to be able to generalize algorithm on unseen data
step 4: groupify train and test sets based on model developed at step 3 then delete group labels
step 5: develop an independent parallel model for each data group

# 5.3.2 PMG-Boosted or PMGB or PMG-TF
Here we tried PMG-2TF and PMG-3TF. As a first try, we divide the data into True and False groups (PMG-2TF). As a second try, data is divided into 3 groups including True and False positive and False negative (PMG-3TF) and steps can be shortened as follows:
step 1: designing initial model on original data set
step 2: labeling data based on misclassified data (T F or T F+ F-)
step 3: develop a model for predicting labels of groups in step 2 to be able to generalize algorithm on unseen data
step 4: groupify train and test sets based on model developed at step 3 then delete T F labels
step 5: develop an independent parallel model for each data group

# 5.4 Deep PMG vs Shallow PMG (One-Layer PMG)
Deep PMG is repeated several times until reaching stopping criteria i.e. after modeling of each group, you can consider each group like initial dataset then find new sub-groups for each group and continue the procedure. However, this approach is more appropriate for PMG-boosted. why? Note that, PMG-boosted do not work based on residuals and is not a boosting algorithm at all. I called it boosted since you can continue groupifying False predictions (and also True predictions) repeatedly to more groups similar what happens in boosting, where we model residuals again and again.

# 5.5 Adjusted PMG to Handle Imbalanced Data 
Traditional classifiers are more biased towards the majority classes. Oversampling and undersampling come with uncertainty by adding (or loosing) data. The existing adjustments like oversampling and undersampling often have difficulty tackling this kind of problem with the class overlapping mainly because they change decision boundaries. 
Here I try new version of PMG to tradeoff between working on imbalanced data and complete oversampling. Since this data is very imbalanced some groups might suffer more from even worse degree of imbalanced data. I tried two custom oversampling "add" and "union" on training data set to slightly improve this issue with a new simple hybrid method of oversampling and applying class weight that uses real data from minor class of other groups to compensate overlapping problem of oversampling.
a) "add": we added all 0 data (minor label) to all groups that is equal to just dividing label 1 to 3 groups. In this version, misclassified data with label 0 are repeated twice
b) "union": at this custom oversampling, we applied union operator of sets to prevent repeated data on each group; each group include the union of original group members of step 4 and minor class i.e. data with label y=0.
There is an interesting article to overcome the problem of overlapping caused by oversampling in "Grouping-based Oversampling in Kernel Space for Imbalanced Data Classification", Jinjun Ren a b, Yuping Wang a, Yiu-ming Cheung c, Xiao-Zhi Gao d, Xiaofang Guo, Pattern Recognition, Volume 133, January 2023, 108992, https://www.sciencedirect.com/science/article/abs/pii/S0031320322004721 . In this article, according to the different possibilities of minority class samples appearing in the overlapping regions in the feature space, oversampling of minor class restricted to region that is far from decision boundaries and the result are superior to main benchmark algorithms, especially in severely imbalanced dataset. It is a great idea, isn't it? I have not tried it in this notebook since I read it after I completed coding. It certainly worth to try it in groups in the future.

# 6. Functions II

In [3]:
# 6. Functions II
def step1_init_model(init_model = initial_model, X_train = X_train, y_train = y_train, X_test = X_test , y_test = y_test):
    print("step1_init_model")
    # calculating TP FN ... based on initial classic model for labeling in next step
    # return F_pred (F predictions) & F0_pred (F predictions when y is from the minor (important) group i.e. y=0)
    model0, y_pred_train, y_pred_test = model_clf(init_model, X_train, y_train) #classic model without my idea
    cond_y0_train, cond_y1_train = y_train==0, y_train==1                        # y_train == 0 ==> (cond_y0_train: True)
    T_pred_train, F_pred_train = y_train ==y_pred_train, y_train !=y_pred_train  # True prediction (T_pred_train)
    T0_pred_train, F0_pred_train = (T_pred_train) & (cond_y0_train),  (F_pred_train) & (cond_y0_train)
    cond_y0_test, cond_y1_test = y_test==0, y_test==1                            #  y_test == 0 ==> cond_y0_test: True
    T_pred_test, F_pred_test = y_test ==y_pred_test, y_test !=y_pred_test        #  pred_test == True
    T0_pred_test, F0_pred_test = (T_pred_test) & (cond_y0_test),  (F_pred_test) & (cond_y0_test)
    return F_pred_train, F_pred_test, F0_pred_train, F0_pred_test # F0 will be needed only if num_g == 3

def step2_labeling(F_pred_train, F_pred_test, F0_pred_train, F0_pred_test, num_g = num_groups, FE_lda = False):
    print("step2_labeling")
    # add label based on misclasified data at the initial model (FE for based on the new label)
    # args: 1) num_g: number of groups & (FE_lda: 2)FE_lda: extract feature for futrue applications
    # n=2 label T (1) & F (0)  
    # n=3 Lalbel T (2) & FP (1) & FN (0)    (FN with more distan from T) (note: pos_label = 0 which minor group) 
    train_sia, test_sia = X_train.copy(), X_test.copy()
    train_sia[tar], test_sia[tar] = y_train, y_test 
    print(num_g)
    if num_g == 2:
        train_sia[tar2]=1; train_sia.loc[F_pred_train, tar2] = 0    
        test_sia[tar2]=1;  test_sia.loc[F_pred_test, tar2] = 0   
    elif num_g ==3:
        train_sia[tar2]=2; train_sia.loc[F_pred_train, tar2] = 1;    train_sia.loc[F0_pred_train, tar2] = 0  
        test_sia[tar2]=2;  test_sia.loc[F_pred_test, tar2] = 1  ;      test_sia.loc[F0_pred_test, tar2] = 0   
    X_train_sia, y_train_sia = train_sia.drop([tar, tar2], axis=1).copy(), train_sia[tar2].copy() 
    X_test_sia, y_test_sia   = test_sia.drop([tar, tar2], axis=1).copy(),  test_sia[tar2].copy()
    if FE_lda == True:
        X_train_sia['my_LDA'], X_test_sia['my_LDA'] = LDA_n(X_train_sia , y_train_sia ,  X_test_sia , y_test_sia , n=1)
    return X_train_sia, y_train_sia, X_test_sia, y_test_sia # X_train_sia=X_train but y_train_sia is changed based on T F

def step3_g(model, X_train_sia, y_train_sia, X_test_sia, y_test_sia):
    print("step3_g")    
    # step 2: fit a model to predict the new label ==> return predictions (& model for possible unseen data in the future)
    model.fit(X_train_sia, y_train_sia)
    y_pred_train_sia, y_pred_test_sia = model.predict(X_train_sia),  model.predict(X_test_sia)
    display(confusion_matrix(y_train_sia, y_pred_train_sia), confusion_matrix(y_test_sia, y_pred_test_sia))
    print('train accuracy is', rnd(100 * accuracy_score(y_train_sia, y_pred_train_sia))\
          , '\t\t test accuracy is',rnd(100 * accuracy_score(y_test_sia, y_pred_test_sia)))
    print('train f1_score is', rnd(100 * f1_score(y_train_sia, y_pred_train_sia, average= "macro")),\
          '\t\t test f1_score is', rnd(100 * f1_score(y_test_sia, y_pred_test_sia, average= "macro")))    
    return model, y_pred_train_sia, y_pred_test_sia # the same as y_pred_g_train, y_pred_g_test

def step4_G0_G1_G2(X_train_sia, y_pred_train_sia, X_test_sia, y_pred_test_sia
                   ,num_g = num_groups, cols_for_g = col_g, oversamp = oversampling):
    print("step4_G0_G1_G2", oversamp)        
    # step4: predictions of step3 ==> grouping
    # first col_g for grouping then col_m for modeling==> parallel modeling

    oversamp = oversampling; cols_for_g = col_g; num_g = num_groups
    # step3: predictions of step2 ==> grouping
    cluster = "Group" # temporary label for grouping (then drop)
    X_train_g, X_test_g = X_train_sia.iloc[:, cols_for_g].copy(), X_test_sia.iloc[:, cols_for_g].copy()
    X_train_g[cluster], X_test_g[cluster] = y_pred_train_sia, y_pred_test_sia
    cond_G_train_dic, cond_G_test_dic, idG_train, idG_test, l = dict(), dict(), dict(), dict(), []
    cond_G_train_dic_without, idG_train_without =  dict(), dict() # without oversampling

    for i in range(num_g):
        cond_G_train_dic_without[i], cond_G_test_dic[i] = X_train_g[cluster]==i ,  X_test_g[cluster] == i
        cond_G_train_dic_over = y_train == 0
        idG_train_without[i],idG_test[i]= X_train_g[cond_G_train_dic_without[i]].index, X_test_g[cond_G_test_dic[i]].index
        idG_y0 = y_train[cond_G_train_dic_over].index
        if oversamp == "add":
            # union plus y0 ==> some y0 repeated twice
            idG_train[i] = list(set(set(idG_train_without[i]).union(set(idG_y0)))) + list(idG_y0)
        elif oversamp == "union":
            idG_train[i] = list(set(set(idG_train_without[i]).union(set(idG_y0))))  # union
        else:
            idG_train[i], idG_test[i] = idG_train_without[i], idG_test[i]
        print(oversamp, i, len(idG_train[i]))
        l.append(X_train_sia.loc[idG_train[i], X_train_sia.iloc[:,col_m].columns])
        l.append(y_train[idG_train[i]])
        l.append(X_test_sia.loc[idG_test[i], X_test_sia.iloc[:,col_m].columns])
        l.append(y_test[idG_test[i]])        
    X_train_g.drop(cluster, axis=1, inplace = True); X_test_g.drop(cluster, axis=1, inplace = True)
    return l

def step5_Parallel_models(z, num_g = num_groups):
    print("step5_Parallel_models")        
    if num_groups == 2:
        X_train_g2_G0, y_train_g2_G0, X_test_g2_G0, y_test_g2_G0\
        , X_train_g2_G1, y_train_g2_G1, X_test_g2_G1, y_test_g2_G1 = z
        
        # G0 (False group)
        TP0_g2_train, FN0_g2_train, FP0_g2_train, TP0_g2_test, FN0_g2_test, FP0_g2_test =\
                        model_clf_2(model_g2_G0, X_train_g2_G0, y_train_g2_G0, X_test_g2_G0, y_test_g2_G0)
        # G1 (True group)
        TP1_g2_train, FN1_g2_train, FP1_g2_train, TP1_g2_test, FN1_g2_test, FP1_g2_test = \
                        model_clf_2(model_g2_G1, X_train_g2_G1, y_train_g2_G1, X_test_g2_G1, y_test_g2_G1)


        TP_g2_train,FN_g2_train,FP_g2_train=TP0_g2_train+TP1_g2_train,FN0_g2_train+FN1_g2_train, FP0_g2_train+FP1_g2_train
        TP_g2_test, FN_g2_test, FP_g2_test = TP0_g2_test+TP1_g2_test, FN0_g2_test+FN1_g2_test, FP0_g2_test+FP1_g2_test

        # why the f-score of second model is lower than first model 
        # while G1 is the group that we predicted True in the first step?
        # It is confusing for me at the first glance; However,
        # the first group include more 0 (minor group) so the more imbalnced data in G1 the harder to reach high f-score
        print("train:",end = "\t"); my_F1_score(TP_g2_train, FN_g2_train, FP_g2_train)
        print("test:",end = "\t"); my_F1_score(TP_g2_test, FN_g2_test, FP_g2_test)
   
    elif num_groups == 3:
        X_train_g3_G0, y_train_g3_G0, X_test_g3_G0, y_test_g3_G0, X_train_g3_G1, y_train_g3_G1,\
        X_test_g3_G1, y_test_g3_G1, X_train_g3_G2, y_train_g3_G2, X_test_g3_G2, y_test_g3_G2 = z

        # G0 (False Negative group)
        #model_LR_paralel_G0 = LogisticRegression(class_weight = dic3(0.78))
        TP0_g3_train, FN0_g3_train, FP0_g3_train, TP0_g3_test, FN0_g3_test, FP0_g3_test =\
                        model_clf_2(model_g3_G0, X_train_g3_G0, y_train_g3_G0, X_test_g3_G0, y_test_g3_G0)

        # G1 (False Positive group)
        model_LR_paralel_G1 = LogisticRegression(class_weight = dic3(0.75))
        TP1_g3_train, FN1_g3_train, FP1_g3_train, TP1_g3_test, FN1_g3_test, FP1_g3_test =\
                                model_clf_2(model_g3_G1, X_train_g3_G1, y_train_g3_G1, X_test_g3_G1, y_test_g3_G1)

        # G2 (True group)
        #model_LR_paralel_G2 = LogisticRegression(class_weight = dic3(0.83), max_iter=200)
        TP2_g3_train, FN2_g3_train, FP2_g3_train, TP2_g3_test, FN2_g3_test, FP2_g3_test =\
                                model_clf_2(model_g3_G2, X_train_g3_G2, y_train_g3_G2, X_test_g3_G2, y_test_g3_G2)

        TP_g3_train, FN_g3_train, FP_g3_train = TP0_g3_train+TP1_g3_train+TP2_g3_train,\
                        FN0_g3_train+FN1_g3_train+FN2_g3_train, FP0_g3_train+FP1_g3_train+FP2_g3_train
        TP_g3_test, FN_g3_test, FP_g3_test = TP0_g3_test+TP1_g3_test+TP2_g3_test,\
                        FN0_g3_test+FN1_g3_test+FN2_g3_test, FP0_g3_test+FP1_g3_test+FP2_g3_test

        print("train:",end = "\t"); my_F1_score(TP_g3_train, FN_g3_train, FP_g3_train) 
        print("test:", end = "\t"); my_F1_score(TP_g3_test, FN_g3_test, FP_g3_test) 

def call_all_steps_of_my_idea(num_g = num_groups, init_model = initial_model, oversampling_step4 = False,
                     X_train = X_train, y_train = y_train, X_test = X_test , y_test = y_test, mdl_g2_G0 = LR(0.82)
                     , mdl_g2_G1 = LR(0.85), mdl_g3_G0 = LR(0.78,3), mdl_g3_G1 = LR(0.75,3), mdl_g3_G2 = LR(0.83,3)):
    global oversampling; oversampling = oversampling_step4  # to change arg of step4 it should be global
    global model_g2_G0, model_g2_G1, model_g3_G0, model_g3_G1, model_g3_G2
    model_g2_G0, model_g2_G1              = mdl_g2_G0, mdl_g2_G1, 
    model_g3_G0, model_g3_G1, model_g3_G2 = mdl_g3_G0, mdl_g3_G1, mdl_g3_G2
    F_pred_train, F_pred_test, F0_pred_train, F0_pred_test = step1_init_model(init_model = initial_model)
    X_train_sia, y_train_sia, X_test_sia, y_test_sia = step2_labeling(F_pred_train, F_pred_test, F0_pred_train,
                                                                      F0_pred_test, num_g = num_groups)
    init_model_g = s3_model_g2 if num_groups == 2 else s3_model_g3  # initial model based on num_groups
    model, y_pred_train_sia, y_pred_test_sia = step3_g(init_model_g, X_train_sia, y_train_sia, X_test_sia, y_test_sia) 
    z = step4_G0_G1_G2(X_train_sia, y_pred_train_sia, X_test_sia, y_pred_test_sia, oversamp=oversampling,
                       num_g = num_groups, cols_for_g = col_g)
    step5_Parallel_models(z, num_g = num_groups)

def EDA(init_model_EDA = LR(.87), X_train = X_train, y_train = y_train):
    model0, y_pred_train, y_pred_test = model_clf(init_model_EDA, X_train, y_train)
    # conditions
    cond_y0_train, cond_y1_train = y_train==0, y_train==1                        # y_train == 0 ==> (cond_y0_train: True)
    T_pred_train, F_pred_train = y_train ==y_pred_train, y_train !=y_pred_train  # True prediction (T_pred_train)
    T0_pred_train, F0_pred_train = (T_pred_train) & (cond_y0_train),  (F_pred_train) & (cond_y0_train)
    cond_y0_test, cond_y1_test = y_test==0, y_test==1                            #  y_test == 0 ==> cond_y0_test: True
    T_pred_test, F_pred_test = y_test ==y_pred_test, y_test !=y_pred_test        #  pred_test == True
    T0_pred_test, F0_pred_test = (T_pred_test) & (cond_y0_test),  (F_pred_test) & (cond_y0_test)
    print(f"% of major class in train: {percent0(y_train)}\t\t % of major class in pred_train: {percent0(y_pred_train)}")
    print(f"% of major class in test: {percent0(y_test)}\t\t % of major class in pred_test: { percent0(y_pred_test)}")
    print("F_train_y=0/F_train_all in %:\t", percent0_cond(F0_pred_train,F_pred_train, df = train)) 
    print("F_test_y=0/F_test_all in %: \t", percent0_cond(F0_pred_test, F_pred_test , test))
    ###
    print("train errors (in percent and num when y=0):\t", percent0_cond(F0_pred_train, cond_y0_train), sum(F0_pred_train))
    #print(percent0_cond(T0_pred_train, cond_y0_train), sum(T0_pred_train))
    print("train errors (in % and num in all data):\t", percent0_cond(F_pred_train), sum(F_pred_train));
    idF_train, idT_train, idF_test, idT_train = F_pred_train.index, T_pred_train.index, \
                                                F_pred_test.index, T_pred_test.index
    # test
    print("test errors (in % & num when y=0):\t", percent0_cond(F0_pred_test, cond_y0_test, df = test), sum(F0_pred_test))
    print("test errors (in % & num in all data):\t", percent0_cond(F_pred_test, df = test), sum(F_pred_test)); 

# modeling with extracted features in Part I without groupifying and printing some useful statistics
EDA()
my_F1_score(2019, 2002, 4273); my_F1_score(500, 477, 1114)
print(np.shape(X_train))  

array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
% of major class in train: 93.71		 % of major class in pred_train: 90.14
% of major class in test: 93.88		 % of major class in pred_test: 89.83
F_train_y=0/F_train_all in %:	 31.87
F_test_y=0/F_test_all in %: 	 29.74
train errors (in percent and num when y=0):	 49.76 2001
train errors (in % and num in all data):	 9.83 6279
test errors (in % & num when y=0):	 48.62 475
test errors (in % & num in all data):	 10.0 1597
Precision, Recall & F1_score:  32.09 50.21 39.15
Precision, Recall & F1_score:  30.98 51.18 38.6
(63882, 72)


# 7. Implementation of PMG-2TF (PMGB, PMG-Boosted, or PMG-TF with 2 Groups)

In [4]:
# 7. Implementation of PMG-2TF (PMGB, PMG-Boosted, or PMG-TF with 2 Groups)
num_groups = 2

# 7.1 PMG-2TF: 2 groups without oversampling
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = LR(0.81), mdl_g2_G1=LR(0.83),\
                         mdl_g3_G0= LR(0.78,3), mdl_g3_G1= LR(0.75,3), mdl_g3_G2= LR(0.83,3))
# trial & error  ==> 0.82 is best weight ==> mdl_g2_G0 = LR(0.82) 
# mdl_g2_G1 might be stronger with higher w to 0 but due to F+ destroy overall performance ==> predicting all y=1 is best
# that is why my_idea failed in this data set
# solution for future:  we should find better more complicated model for them to predict a few 0s with low number of FP
# for now  we will try oversampling
######################################################################

# 7.2 PMG-2TF:2 groups with 2 types of oversampling ==> oversampling is not helpful
# 7.2.1 PMG-2TF:2 groups with "union" oversampling
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4 = "union", mdl_g2_G0 = LR(0.77), mdl_g2_G1=LR(0.77),\
                         mdl_g3_G0= LR(0.78,3), mdl_g3_G1= LR(0.75,3), mdl_g3_G2= LR(0.83,3))
# The results of increasing weight of 0 is much better after oversampling! however FP to TP ratio is still not acceptable 
# so, predictions of all data of group 1 as 1 is still the best result for G1; 
# however, trying other models probably will be helpful; for now let's try another type of oversampling
# 7.2.2 PMG-2TF:2 groups with "add" oversampling
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4 = "add", mdl_g2_G0 = LR(0.69), mdl_g2_G1=LR(0.70),\
                         mdl_g3_G0= LR(0.78,3), mdl_g3_G1= LR(0.75,3), mdl_g3_G2= LR(0.83,3))

step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
2
step3_g


array([[ 4862,  1417],
       [ 6775, 50828]], dtype=int64)

...

array([[ 568,   69],
       [1958,  378]], dtype=int64)

train accuracy is 51.59 		 test accuracy is 31.82
train f1_score is 64.25 		 test f1_score is 35.92


array([[ 5240,  2802],
       [ 1833, 48996]], dtype=int64)

array([[   54,   286],
       [  492, 12166]], dtype=int64)

train accuracy is 92.13 		 test accuracy is 94.01
train f1_score is 69.34 		 test f1_score is 12.19
train:	Precision, Recall & F1_score:  57.19 78.76 66.26
test:	Precision, Recall & F1_score:  20.25 63.66 30.72


In [5]:
# 7.3 try Random Forest Classifier and support vector machine classifier for PMG-2TF
dd_knn ={0:0.964, 1:0.036}
knn1 = KNeighborsClassifier(n_neighbors=11)
RF1 = RFC(class_weight=dic2(.91),max_depth=3, max_leaf_nodes=17, max_features=9, random_state=0 )
svm1 = SVC(kernel="rbf", gamma=0.5, C=1.0,class_weight=dic2(.91))

# PMG-2TF: 2 groups with RF or Random Forest Classifier
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81), mdl_g2_G1=RF(0.83),\
                         mdl_g3_G0= RF(0.78,3), mdl_g3_G1= RF(0.75,3), mdl_g3_G2= RF(0.83,3))

step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
2
step3_g


array([[ 4862,  1417],
       [ 6775, 50828]], dtype=int64)

...

array([[ 431,  206],
       [ 847, 1489]], dtype=int64)

train accuracy is 64.84 		 test accuracy is 64.58
train f1_score is 46.54 		 test f1_score is 45.01


array([[   55,  1361],
       [  141, 50688]], dtype=int64)

array([[   18,   322],
       [   62, 12596]], dtype=int64)

train accuracy is 97.13 		 test accuracy is 97.05
train f1_score is 6.82 		 test f1_score is 8.57
train:	Precision, Recall & F1_score:  35.0 45.66 39.63
test:	Precision, Recall & F1_score:  33.06 45.96 38.46


In [6]:
# 7.4 PMG-2TF with heterogeneous classifiers for each group (RF for G0 and SVM with different kernels for G1)
# advantage of grouping ==> possible to try different classifiers for example svc o(n^3) for smaller groups 

# 7.4.1 PMG-2TF: RF for G0 and SVM (kernel: RBF) for G1, the smaller group 
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81), mdl_g2_G1=SV(0.85),\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))


# train accuracy is 65.13 		 test accuracy is 65.16
# train f1_score is 46.53 		 test f1_score is 45.14
# train accuracy is 97.22 		 test accuracy is 97.28
# train f1_score is 2.02 		 test f1_score is 1.12
# train:	Precision, Recall & F1_score:  35.27 44.27 39.26
# test:	Precision, Recall & F1_score:  33.7 43.91 38.13

# 7.4.2 PMG-2TF: RF for G0 and SVM (kernel: poly, degree=3) for G1
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81),\
                          mdl_g2_G1 = SVC(kernel="poly", gamma=0.5, C=1.0,class_weight=dic2(.85)),\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.4.3 PMG-2TF: RF for G0 and SVM (kernel:poly, degree=4) for G1
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81),\
                          mdl_g2_G1 = SVC(kernel="poly", degree =4, gamma=0.5, C=1.0,class_weight=dic2(.86)),\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.4.4 PMG-2TF: RF for G0 and SVM (kernel:poly, degree=5) for G1
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81),\
                          mdl_g2_G1 = SVC(kernel="poly", degree =5, gamma=0.5, C=1.0,class_weight=dic2(.86)),\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.4.5 PMG-2TF: RF for G0 and SVM (kernel:linear) for G1
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = RF(0.81),\
                          mdl_g2_G1 = SVC(kernel="linear", gamma=0.5, C=1.0,class_weight=dic2(.86)),\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
2
step3_g


array([[ 4862,  1417],
       [ 6775, 50828]], dtype=int64)

...

array([[ 431,  206],
       [ 847, 1489]], dtype=int64)

train accuracy is 64.84 		 test accuracy is 64.58
train f1_score is 46.54 		 test f1_score is 45.01


array([[   42,  1374],
       [  128, 50701]], dtype=int64)

array([[   12,   328],
       [   40, 12618]], dtype=int64)

train accuracy is 97.13 		 test accuracy is 97.17
train f1_score is 5.3 		 test f1_score is 6.12
train:	Precision, Recall & F1_score:  34.93 45.34 39.46
test:	Precision, Recall & F1_score:  33.31 45.34 38.4


In [7]:
# 7.5 PMG-2TF: AdaBoost classifier (AdaBoostClassifier) and Gaussian Naive Bayes (GaussianNB)
adb = ABC(n_estimators=200, random_state=0 ) # AdaBoostClassifier
adb0 = ABC(n_estimators=200, random_state=0, base_estimator= LR(0.73) )
adb1 = ABC(n_estimators=200, random_state=0, base_estimator= LR(0.97) )

# 7.5.1 AdaBoostClassifier
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = adb,\
                          mdl_g2_G1 = adb,\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.5.2 AdaBoostClassifier with custom base_estimator
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4="False"\
                          , mdl_g2_G0 = adb0, mdl_g2_G1 = adb1,\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.5.3 AdaBoostClassifier with custom base_estimator with different class_weights
adb0 = ABC(n_estimators=200, random_state=0, base_estimator= LR(0.66) )
adb1 = ABC(n_estimators=200, random_state=0, base_estimator= LR(0.90) )

call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4="union"\
                          , mdl_g2_G0 = adb0, mdl_g2_G1 = adb1,\
                         mdl_g3_G0= SV(0.78,3), mdl_g3_G1= SV(0.75,3), mdl_g3_G2= SV(0.83,3))

# 7.5.4 GaussianNB
gnb = GaussianNB()
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False\
                          , mdl_g2_G0 = gnb, mdl_g2_G1 = gnb,\
                         mdl_g3_G0= RF(0.78,3), mdl_g3_G1= RF(0.75,3), mdl_g3_G2= RF(0.83,3))

step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
2
step3_g


array([[ 4862,  1417],
       [ 6775, 50828]], dtype=int64)

...

array([[ 219,  418],
       [ 235, 2101]], dtype=int64)

train accuracy is 77.66 		 test accuracy is 78.04
train f1_score is 40.5 		 test f1_score is 40.15


array([[  228,  1188],
       [ 1467, 49362]], dtype=int64)

array([[   56,   284],
       [  429, 12229]], dtype=int64)

train accuracy is 94.92 		 test accuracy is 94.51
train f1_score is 14.66 		 test f1_score is 13.58
train:	Precision, Recall & F1_score:  32.17 27.68 29.76
test:	Precision, Recall & F1_score:  29.29 28.15 28.71


# 8. Implementation of PMG-3TF (PMGB, PMG-Boosted, or PMG-TF with 3 Groups)


In [8]:
# 8. Implementation of PMG-3TF (PMGB, PMG-Boosted, or PMG-TF with 3 Groups)
num_groups =3

# 8.1 PMG-3TF without oversampling
# 8.1.1 PMG-3TF without oversampling with Linear regression
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = LR(0.82), mdl_g2_G1=LR(0.85),\
                         mdl_g3_G0= LR(0.78,3), mdl_g3_G1= LR(0.75,3), mdl_g3_G2= LR(0.83,3))
# 8.1.2 PMG-3TF without oversampling with minor difference in class_weight 
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4=False, mdl_g2_G0 = LR(0.82), mdl_g2_G1=LR(0.88),\
                         mdl_g3_G0= LR(0.83,3), mdl_g3_G1= LR(0.75,3), mdl_g3_G2= LR(0.85,3))
# F-score: 38.38


step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
3
step3_g


array([[ 1166,    18,   817],
       [  572,  3701,     5],
       [13699,  1993, 41911]], dtype=int64)

...

array([[390,  64],
       [722, 283]], dtype=int64)

train accuracy is 47.81 		 test accuracy is 46.13
train f1_score is 51.63 		 test f1_score is 49.81


array([[   43,   780],
       [  124, 41786]], dtype=int64)

array([[   14,   197],
       [   40, 10364]], dtype=int64)

train accuracy is 97.88 		 test accuracy is 97.77
train f1_score is 8.69 		 test f1_score is 10.57
train:	Precision, Recall & F1_score:  34.08 45.96 39.14
test:	Precision, Recall & F1_score:  32.76 46.67 38.5


it seems we need less 0 prediction in our model (this happened due to maximizing F-score via dd in class_weight arg since missing in predicting 0 penaltize more) following code show, unlike my first comment, while 0 are much more important for f score the errors are just twice. so you may change predictor in a way that result in more 0s to maximze f-score


ruslts of that simple model shows that ==> 2002 mistakes for y=0 with 50.21% accuracy for y=0 
note: you may think that 50% accuracy is useless at first glance, since it seems like random; however,
that 50% accuracy is not very bad score for minor group in imbalanced data; because 
it is hard to detect if it is from minor class or not.

In [9]:
# 8.2 PMG-2TF:3 groups with 2 types of oversampling ==> oversampling is not helpful
# 8.2.1 union
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4="union", mdl_g2_G0 = LR(0.82), mdl_g2_G1=LR(0.88),\
                         mdl_g3_G0= LR(0.81,3), mdl_g3_G1= LR(0.73,3), mdl_g3_G2= LR(0.84,3))
# 8.2.2 add
call_all_steps_of_my_idea(num_g = num_groups, oversampling_step4="add", mdl_g2_G0 = LR(0.82), mdl_g2_G1=LR(0.88),\
                         mdl_g3_G0= LR(0.77,3), mdl_g3_G1= LR(0.6,3), mdl_g3_G2= LR(0.88,3))
# so, let’s forget about this type of oversampling y=0
# The results are disappointing; however; I believe that it might work on another data set that meet conditions required.

step1_init_model


array([[ 2020,  2001],
       [ 4278, 55583]], dtype=int64)

array([[  502,   475],
       [ 1122, 13872]], dtype=int64)

train accuracy is 90.17 		 test accuracy is 90.0
train f1_score is 39.15 		 test f1_score is 38.6
step2_labeling
3
step3_g


array([[ 1166,    18,   817],
       [  572,  3701,     5],
       [13699,  1993, 41911]], dtype=int64)

...

array([[434,  20],
       [977,  28]], dtype=int64)

train accuracy is 67.1 		 test accuracy is 31.67
train f1_score is 80.07 		 test f1_score is 46.54


array([[ 7052,   990],
       [ 7340, 34570]], dtype=int64)

array([[  97,  114],
       [1852, 8552]], dtype=int64)

train accuracy is 83.32 		 test accuracy is 81.48
train f1_score is 62.87 		 test f1_score is 8.98
train:	Precision, Recall & F1_score:  51.12 91.69 65.64
test:	Precision, Recall & F1_score:  12.26 77.99 21.19


# 9. Implementation of PMGC (PMG-Clustered or General PMG) with K Groups
Here, we will apply k-mean for clustering, you may try other clustering algorithms like hierarchical clustering. We assume number of clusters is 3 to be comparable with PMG-boosted; However, for better performance you should find best number of clusters.

In [10]:
# 9. Implementation of PMGC (PMG-Clustered or General PMG) with K Groups
df_train = X_train.iloc[:, col_m].copy(); df_train[tar] = y_train

# 9.1 first try: cluster using all data
kmeans = KMeans(n_clusters = 3, random_state = 0, n_init='auto') # we assume number of clusters is 3; for better performance 
kmeans.fit(X_train.iloc[:, col_m]) # kmeans.cluster_centers_
pred_train, pred_test = kmeans.fit_predict(X_train.iloc[:, col_m]), kmeans.fit_predict(X_test.iloc[:, col_m])
display(pd.Series(pred_train).value_counts(), pd.Series(pred_test).value_counts())
X_train_sia, X_test_sia = X_train.iloc[:, col_m].copy(), X_test.iloc[:, col_m].copy()
X_train_sia[tar2], X_test_sia[tar2]  = pred_train, pred_test

X_train_g0, X_test_g0 = X_train_sia.loc[X_train_sia[tar2]==0,:], X_test_sia.loc[X_test_sia[tar2]==0,:]
X_train_g1, X_test_g1 = X_train_sia.loc[X_train_sia[tar2]==1,:], X_test_sia.loc[X_test_sia[tar2]==1,:]
X_train_g2, X_test_g2 = X_train_sia.loc[X_train_sia[tar2]==2,:], X_test_sia.loc[X_test_sia[tar2]==2,:]
y_train_g0, y_train_g1, y_train_g2 = y_train[X_train_g0.index], y_train[X_train_g1.index], y_train[X_train_g2.index]
y_test_g0, y_test_g1, y_test_g2 = y_test[X_test_g0.index], y_test[X_test_g1.index], y_test[X_test_g2.index]
z = [X_train_g0, y_train_g0, X_test_g0, y_test_g0, 
     X_train_g1, y_train_g1, X_test_g1, y_test_g1, 
     X_train_g2, y_train_g2, X_test_g2, y_test_g2]

# 9.2 modeling with linear regression
mdl_g3_G0, mdl_g3_G1, mdl_g3_G2 = LR(0.78,3), LR(0.85,3), LR(0.81,3)
model_g3_G0, model_g3_G1, model_g3_G2 = mdl_g3_G0, mdl_g3_G1, mdl_g3_G2
step5_Parallel_models(z, num_g = 3)

0    27247
1    25286
2    11349
dtype: int64

1    9587
2    4564
0    1820
dtype: int64

step5_Parallel_models


array([[  404,   923],
       [  652, 25268]], dtype=int64)

array([[  93,   95],
       [ 162, 1470]], dtype=int64)

...

array([[ 279,  293],
       [ 686, 8329]], dtype=int64)

train accuracy is 89.49 		 test accuracy is 89.79
train f1_score is 37.44 		 test f1_score is 36.3


array([[ 575,  526],
       [1057, 9191]], dtype=int64)

array([[  58,  159],
       [ 109, 4238]], dtype=int64)

train accuracy is 86.05 		 test accuracy is 94.13
train f1_score is 42.08 		 test f1_score is 30.21
train:	Precision, Recall & F1_score:  33.21 44.12 37.89
test:	Precision, Recall & F1_score:  31.0 44.01 36.38


In [11]:
# 9.3 modeling with combination of LR and RF
mdl_g3_G0, mdl_g3_G1, mdl_g3_G2 = LR(0.78,3), RF(0.75,3), LR(0.81,3)
model_g3_G0, model_g3_G1, model_g3_G2 = mdl_g3_G0, mdl_g3_G1, mdl_g3_G2
step5_Parallel_models(z, num_g = 3)

step5_Parallel_models


array([[  404,   923],
       [  652, 25268]], dtype=int64)

array([[  93,   95],
       [ 162, 1470]], dtype=int64)

train accuracy is 94.22 		 test accuracy is 85.88
train f1_score is 33.91 		 test f1_score is 41.99


array([[  693,   900],
       [ 1113, 22580]], dtype=int64)

array([[ 208,  364],
       [ 295, 8720]], dtype=int64)

train accuracy is 92.04 		 test accuracy is 93.13
train f1_score is 40.78 		 test f1_score is 38.7


array([[ 575,  526],
       [1057, 9191]], dtype=int64)

array([[  58,  159],
       [ 109, 4238]], dtype=int64)

train accuracy is 86.05 		 test accuracy is 94.13
train f1_score is 42.08 		 test f1_score is 30.21
train:	Precision, Recall & F1_score:  37.21 41.58 39.27
test:	Precision, Recall & F1_score:  38.81 36.75 37.75


# 10. PMG: Drawbacks, Challenges and Conditions
The main challenge of MPG is that the label just includes information about main task and do not include any information about data source! For example, Kaggle Diabetes data set implies that somebody suffer from diabetes when label is 1 but we have no idea about type of diabetes! How should we separate diabetes type-I from type-II. The other challenge is that if this data set really consist of different groups? Although we can try some techniques like clustering to guess, I do not have solid response to that question without domain knowledge in special datasets and this is one of the drawbacks of presented idea. Another important challenge is how to find group members especially, in many cases, when we do not even know how many groups are in the dataset. Having no label for these groups, we had to find these groups based on intrinsic characteristics of data set. However, elbow method might be helpful to find number of clusters. Even if we overcome these problems, there are many other challenges like grouping error. Although we cannot groupify data to pure groups (for example group 1 just including diabetes type-I), the idea is that each group will be much more pure than original data. Shortly, PMG is useful only if the following two conditions meet: 1) Dataset originally consist of multiple clusters (like diabetes example that explained) 2) finding an appropriate grouping model to split data; the difference in behavior of groups are so high that can compensate the grouping error. The first one is intrinsic characteristic of dataset while the latter is our job to find. So, PMG is not appropriate for any data set that does not meet first condition. Please write a comment if you have any useful idea or metric to find out whether a data set consists of several groups or not before applying PMG.

# 11. Another Viewpoint: Optimization Perspective
Consider special situation where all of the models of same level use the same model, for example logistic regression (or linear regression if it is regression task). In this situation, applying 2 (or 3 or more) different models to each group is equivalent to using one model with updating weights using a grouping optimizer; So, grouping SGD or grouping Adam is a very special cases of PMG where exactly the same models with the same parameters are used for all groups. Research in that field is so narrow that I hardly find something about grouping SGD and I have yet to find any paper about grouping Adam and I believe it has not available in literature. However, the algorithms presented are completely different with different results. Refer to: "Multi-Level Local SGD: Distributed SGD for Heterogeneous Hierarchical Networks", Timothy Castiglia, Anirban Das, Stacy Patterson, 2023 ICLR, https://openreview.net/forum?id=C70cp4Cn32 and "Adaptive Worker Grouping For Communication-Efficient and Straggler-Tolerant Distributed SGD", Feng Zhu, Jingjing Zhang, Osvaldo Simeone, Xin Wang, 2022, Computer Science, Information Theory, https://arxiv.org/abs/2201.04301 . Grouping sgd is great option to improve optimization when we have multiple sources of data as explained. However, it is very limited! It is hard to handle imbalanced data with class weight argument in this scenario since the degree of imbalance in data might be significantly different in each group. In addition, and more importantly, it cannot leverage the freedom of using different models for each group, for example applying SVM or deep NN for a group with lowest samples which is possible in PMG. Furthermore, it is not as effective as PMG in the combination with boosting as it will be explained in future work section. However, it is useful idea and very special case of PMG.

# 12. What Distinguish PMG from Other Methods in Literature?
I was not sure if my idea is really new; so, I tried to search expression like "group-based modeling" and "grouping regression" in literature but I couldn't find the same idea in recent papers. Some of similar but different ideas are followed. You can think of this section as literature review.

# 12.1 Boosting
Please do not confuse multi-layer iterative PMG with boosting where boosting-based algorithms grow in length serially and input data of each model is the residuals of previous model while PMG grows in width parallelly.

# 12.2 Parallel Processing or Parallelization
Data parallelism emphasizes the distributed (parallel) nature of the data and splits a single model onto different GPUs as opposed to the processing (task parallelism). In parallel processing, the main task is broken up into multiple, smaller tasks to be processed individually that makes distributed processing possible. Here, on the other hand, focus is on finding a better model not faster execution, still we can leverage the advantages of distributed processing in modeling each subset; however, it is not as much optimal as in parallel processing from the execution speed since instead of dividing data into equal subsets, in my idea both number of groups and number of members in each group are determined based on properties of the dataset.

# 12.3 Two-Step Classification
In two-step classification, for classifying 3 class problem as it is presented in "A Comparison of 2 Step Classification with 3-Class Classification for Webpage Classification", 2022 13th International Conference on Information and Communication Technology Convergence (ICTC), https://ieeexplore.ieee.org/document/9952595, 2 primary models (here 2 binary classification) work serially and perform binary classification at each level and it is very useful idea when there are 3 classes including one outstandingly easy group to be classified while other two classes are very similar, hard to classify.  In the PMG, on the other hand, models work on n parallel groups. In fact, Two-step classification involves breaking down a complex task into 2 simpler sub-tasks which one is relatively easier to classify called "easy" or "simple" the other is harder and needs more complex model while, in PMG, the underlying reason of groupifying is potential multiple data sources and no group is necessarily more complicated than the others. In addition, data is not divided into 2 groups, instead, divided based on the original number of sources generated the data which is an intrinsic property of the data; so, the number of classes is initially unknown unless we have domain knowledge. Furthermore, PMG is general approach that does not limited to classification tasks while two-step classification is limited and is not applicable even for binary classification tasks like current dataset.

# 12.4 Grouping Based Classification
In grouping-based classification, data is groupify and independent feature selection is applied to each group. In fact, this is the closest idea to PMG; However, at least as much as I understand, since the authors have no intuitive understanding about why different groups can be modeled with different features, they restrict their idea on feature selection. Although these works have great impact on ML with outstanding results, they can be considered as very special scenario of PMG; while PMG can dictate its superior power by freedom in modeling. In addition, idea underlying PMG make it possible to apply domain knowledge in some cases to find number of groups. "Multilabel Classification With Group-Based Mapping: A Framework With Local Feature Selection and Local Label Correlation", Jianghong Ma; Bernard Chi Yuen Chiu; Tommy W. S. Chow, IEEE Transactions on Cybernetics, Volume 52, 2022 emphasizes that different subsets of samples may share different feature selection weights and different label correlations and present a novel framework with local feature selection and local label correlation, where  instances can be clustered into different groups, and the feature selection weights and label correlations can only be shared by instances in the same group. "A N-binary Classification and Grouping-based Approach to Improve the Performance of Anomaly Detection", Omkar Shende, R. K. Pateriya & Priyanka Verma, Arabian Journal for Science and Engineering, Volume 47, pages 1275–1287, 2022 proposed an anomaly-based IDS with novel hybrid ensemble classification method based on grouping of the network traffic. After grouping of network traffic, wrapper-based sequential feature selection (SFS) with random forest (RF) classifier is used to select optimal features and perform classification in each group. 

# 12.5 Grouping Regression
There are just name similarities and this old term is not directly related to our topic and articles referring to grouping regression mainly focus on transformation of continuous feature, discretizing the continuous feature into multiple groups and then applying transformation, target encoding for example, for linear algorithms. ML algorithms are not powerful enough to follow every behavior and each algorithm has its own restrictions.  Linear regressions can follow linear trend in data like X^3 and that is why correlation is very important criteria in feature selection for these algorithms; however, they are not able to follow U-shape functions like X^2 without transformation. Do you remember logistic regression formula?  Similarly, logistic regression is able to follow step function which are binary version of S-shape functions like X^3 but not pulse functions which are binary version of U-shape functions. Although tree-based algorithm does not suffer from that problem, in many cases, they are not able to follow if number of features to tree-depth ratio is big. That is why data transformation is very important. For example, this feature is hard to be useful in data and need reprocessing like target encoding of groups. 

# 13. Advantages of PMG
a) increased accuracy: The main advantage of this approach is to increase the accuracy and any required evaluation metric in datasets consists of multiple sources. 
b) distributed processing: Parallel processing characteristic of this approach increases the speed execution in one computer and also enable distributed processing in several systems but it is slower than data parallelism method. Some algorithms like deep neural networks and Support vector machines are not suitable to process large training set; for example, standard SVM classifier computational complexity can reach O(n^3); However, groupifying data make it possible to apply these algorithms on smaller subsets. 
c) Deep PMG (iterating ability of PMG inside PMG): Another advantage of this approach is that, like boosting, you can continue, especially the PMG-boosted, as many times as needed i.e. you can divide predictions of new model again to new groups, for example, at the first step divide data to T and F and find appropriate model for each group (first layer of PMG). Note that, even in the T group, the prediction will not be 100 percent due to potential grouping error. In the second step, consider each group as a new data set and groupify each group again into T and F then model new subgroups (second layer of PMG). You can continue these layers deeper in parallel of each other until stopping criteria meet. However, similar to boosting based algorithms, the deeper we go, the more prone to overfit.
d) general approach: PMG is general approach that can be applied both on regression and on classification tasks. There is no major difference in applying the proposed algorithm on regression. You should first divide data into multiple groups. The clustering algorithm might be more appropriate to find different sources that explained; however, you can also divide based on the residual of modeling (similar to T and F). if you want to divide into 2 groups a predefined threshold might be calculated based on trial and error. Also, some clustering algorithms can estimate number of optimal groups.
e) ability to apply in hybrid methods: One of the advantages of PMG-boosted is its ability to be combined with boosting-based algorithms to create significantly powerful method. 

# 14. Conclusion
The results do not show sensible improvement in F-score since dataset is not appropriate and data preprocessing it the most important phase in this dataset and reconsidering outlier manipulation and missing values (NA) handling, for example with KNN, are keys to increase the performance of modeling. However, I belive that the idea behind this notebook is theoretically so strong that various shalow and deep version of PMG and their combinations with bagging and boosting will revolutionize the performance of ML algorithms in the cases that meet required conditions to apply. We will see in the futerue. 

# 15. Future Works: Hybrid PMG with Boosting & Bagging
The combination of PMG with boosting and bagging  is the proposed method in such a way that first apply PMG (groupify data then model each group with a simple algorithm which is not necessarily a tree-based one. The whole PMG block will work like a boosting base model i.e. the residuals of first block will be the input of the second block. In the second block, we will again groupify each group then apply another simple model which is not necessarily the same. The bagging will be applied in feature selection. The advantages of such a hybrid approach are as follow:
a) Includes all advantages of boosting and bagging
b) All advantages of my idea of groupifying the data
c) improved version of boosting that is less prone to overfit which is a major concern in boosting by decreasing iterations of boosting needed due to grouping. In addition, all groups are not needed to go as deep as the others based on predefined stopping criteria 
d) All simple models are not necessarily the same! So, you can leverage the advantages of multiple simple sub-models. For example, in the odd blocks use decision tree while even block include linear regression. This may result in potential decrease in the number of iterations needed and also in increase of evaluation metrics like accuracy since each simple sub-model can compensate the disadvantage of the previous one that applies in its residuals. The last advantage is not limited to this idea and can be applied to any boosting algorithm. 

# 15.1 Hybrid (Shallow) PMGC with Boosting & Bagging
In one hand, in hybrid shallow PMGC (PMG-clustered or general PMG) with boosting and bagging, Boosting should be limited to groups without groupifying any more i.e. if we have 7 clusters, for example, a shallow PMGC will be followed by 7 parallel completely dependent boosting ML algorithms with bagging feature selection for residuals of each group. Although, PMGC is theoretically able to be combine with boosting in all layers, it is not recommended to groupify in all layers in order to avoid overfitting unless the number of groups on each iteration is very small.

# 15.2 Hybrid (Deep-Shallow) PMGB with Boosting & Bagging
On the other hand, hybrid shallow PMG-Boosted or PMGB or PMG-TF with boosting and bagging is more flexible method. Hybrid shallow PMGB with boosting and bagging is exactly the same as the previous method except for PMGC is replaced by PMGB. However, in the deep hybrid version, you can repeat groupifying and modeling in any layer. If boosting is too deep consisting of many simple base models, it is not recommended to apply PMGB on all iterations but, based on dataset, apply just on, for example, even layers of boosting or randomly with probability of 20 percent on all layers.